# Evaluate model in paper Machine Learning for Trading
Link [paper](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3015609)

In [1]:
import os
from pathlib import Path
path = Path(os.getcwd())
os.chdir(path.parent.absolute())

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import pandas_ta as ta
from stable_baselines3.dqn import DQN
import gym
import matplotlib.pyplot as plt
from stable_baselines3.common.evaluation import evaluate_policy
import mt4_hst
import stock_env
from stock_env.utils import *
from stock_env.envs.vn_stock_env import VietnamStockEnv
from stable_baselines3.common.env_checker import check_env
%matplotlib inline

In [2]:
FPT = mt4_hst.read_hst("stock_env/datasets/FPT1440.hst")
FPT = FPT[FPT['time'] >= '2012-01-01']
env = VietnamStockEnv(
    df=FPT,
    max_trade_lot=5,
    max_lot=10,
    init_cash=100e3)
check_env(env)
model = DQN.load("log/dqn_VietnamStock", env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
# run model to get detailed information in the enviroment
done = False
obs = env.reset()
obs.shape
# while not done:
#     obs, _, done, _ = env.step(env.action_space.sample())
# # get data
# history_df = env.get_history()

In [ ]:
# run model to get detailed information in the enviroment
done = False
obs = env.reset()
while not done:
    action, _ = model.predict(obs, deterministic=True)
    obs, _, done, _ = env.step(action)

# get data
history_df = env.get_history()
history_df.to_csv('temp/history.csv', index=False)

In [ ]:
columns = 'time close delta_shares quantity portfolio_value nav cash'.split()
df = pd.read_csv('temp/history.csv', index_col=0)
df = df[columns]
# buy = df[df['delta_shares'] < 0]

In [ ]:
FPT = mt4_hst.read_hst("stock_env/datasets/FPT1440.hst")
FPT = FPT[FPT['time'] >= '2012-01-01']
FPT.sort_values(by='time', inplace=True)
FPT = FPT.reset_index(drop=True)

FPT.ta.rsi(append=True)
FPT.ta.tsignals(FPT['close'] > ta.sma(FPT['close'], 50), append=True)
# FPT.ta.log_return(append=True)
# FPT.ta.percent_return(append=True)
# FPT.ta.roc(append=True)
# FPT.ta.stoch(append=True)
# FPT.ta.stochrsi(append=True)
# FPT.ta.tsi(append=True)

# FPT.describe()
FPT.head()

In [ ]:
remove_col = set('time open high close low volume'.split())
cols = list(set(FPT.columns).difference(remove_col))
features = FPT[cols]
features.dropna(inplace=True)
features = features.reset_index(drop=True)
features.head()

In [ ]:
import numpy as np
obs = features.iloc[1].values
np.append(obs, float(0)).astype(np.float32)

In [ ]:
FPT.ta.strategy("performance")
FPT['time open high low close volume RSI_14 AO_5_34'.split()].dropna()

In [ ]:
df = pd.read_csv('temp/history.csv')
df.sort_values(by='time', inplace=True)
df.index = pd.to_datetime(df['time'])
portfolio_value = df['portfolio_value']

In [ ]:
plot_trade_log_v2(df)

In [ ]:
create_performance(portfolio_value.pct_change())

In [ ]:
cols = 'time close delta_shares quantity portfolio_value cash nav'.split()
df[cols][df['delta_shares'] != 0]

In [ ]:
help(df.ta.tsignals)